In [5]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, text

# Create an SQLite engine (in-memory database)
engine = create_engine('sqlite:///:memory:', echo=False)
metadata = MetaData()

# Define the sales table
sales = Table('sales', metadata,
              Column('Salesperson', String, nullable=False),
              Column('Year', Integer, nullable=False),
              Column('Revenue', Float, nullable=False))

# Create the table in the database
metadata.create_all(engine)

# Insert data
with engine.connect() as conn:
    conn.execute(sales.insert(), [
        {'Salesperson': 'John', 'Year': 2020, 'Revenue': 5000},
        {'Salesperson': 'John', 'Year': 2021, 'Revenue': 6000},
        {'Salesperson': 'Alice', 'Year': 2020, 'Revenue': 7000},
        {'Salesperson': 'Alice', 'Year': 2021, 'Revenue': 8000}
    ])
    print("Data inserted successfully.")

# Verify the inserted data
with engine.connect() as conn:
    result = conn.execute(text('SELECT * FROM sales'))
    rows = result.fetchall()
    
    print("Inserted data:")
    for row in rows:
        print(row)


# Querying and pivoting (manual pivot with case statements)
with engine.connect() as conn:
    result = conn.execute(text('''
        SELECT Salesperson,
               SUM(CASE WHEN Year = 2020 THEN Revenue ELSE 0 END) AS "2020",
               SUM(CASE WHEN Year = 2021 THEN Revenue ELSE 0 END) AS "2021"
        FROM sales
        GROUP BY Salesperson
    '''))
    
    # Fetch and print results
    rows = result.fetchall()
    if rows:
        for row in rows:
            print(row)
    else:
        print("No results found.")


Data inserted successfully.
Inserted data:
No results found.


In [9]:
import pandas as pd

def find_correlated_products(corr_table, product, correlated=None):
    if correlated is None:
        correlated = set()
    
    # Add the product itself to the correlated set
    correlated.add(product)
    
    # Find correlated products in the row (index)
    if product in corr_table.index:
        for col in corr_table.columns:
            if corr_table.at[product, col] != 0 and col not in correlated:
                find_correlated_products(corr_table, col, correlated)

    # Find correlated products in the column
    if product in corr_table.columns:
        for idx in corr_table.index:
            if corr_table.at[idx, product] != 0 and idx not in correlated:
                find_correlated_products(corr_table, idx, correlated)
    
    return list(correlated)


In [10]:
# Example correlation table (non-symmetrical)
data = {
    'A': [0, 1, 0, 0],
    'B': [1, 0, 0, 1],
    'C': [0, 0, 0, 0],
    'D': [0, 1, 0, 0]
}

# Non-symmetric table
corr_table = pd.DataFrame(data, index=['A', 'B', 'D', 'E'])

# Example usage
result = find_correlated_products(corr_table, 'A')
print(result)

['E', 'B', 'A', 'D']
